In [1]:
import nltk
import numpy as np
import json
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu

[nltk_data] Downloading package punkt to /home/grad3/hle/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
references = np.load('./data/references_from_0_to_1000.npy', allow_pickle=True)
expressions_1 = np.load('./data/top3_exps_from_0_to_1000.npy', allow_pickle=True)

In [13]:

tokenized_ref = []
ref_1 = [r[0] for r in references[500]]
print(ref_1)
for r in ref_1:
    lis=word_tokenize(r.lower())
    tokenized_ref.append(lis)
print(tokenized_ref)
print(expressions_1[500])
print(word_tokenize(expressions_1[500][1]))

['Vase furthest left', 'far left vase', 'far left vase']
[['vase', 'furthest', 'left'], ['far', 'left', 'vase'], ['far', 'left', 'vase']]
['table' 'gray sitting flowers' 'wall']
['gray', 'sitting', 'flowers']


In [4]:
BLEUscore = nltk.translate.bleu_score.sentence_bleu(tokenized_ref, ['shirt'])
print("BLEU:",BLEUscore)

BLEU: 0.36787944117144233


/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


In [26]:
bleu_score = []
files = [0,1000,2000,3000,4000]
for start in files:
    references = np.load(f'./data/references_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    expressions = np.load(f'./data/generated_expressions/test/generated_expression_probabilisticutterance_lstm_adjusted/top3_exps_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    for ref, exps in zip(references, expressions):
        item_reference = [word_tokenize(r[0].lower()) for r in ref]
    #     print(item_reference)
    #     print(exps)
    #     print('$$$$$$$$$$$')

        best_matched = max([sentence_bleu(item_reference, word_tokenize(exp)) for exp in exps])#[0]
        bleu_score.append(best_matched)
print(len(bleu_score))

/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


5000


In [27]:
x = [score for score in bleu_score if score != 0]
print(np.average(x), np.std(x))
print(np.average(bleu_score), np.std(bleu_score))
print(len(x))
# 0.3939225254870882 0.3270738202276801
# 0.1983005993302002 0.3043752780752566

0.6693761222323064 0.24604247525179773
0.4392446114088394 0.37524437314968107
3281


### CALCULATE ROUGE SCORE


In [44]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)

In [5]:
references = np.load('./data/references_from_0_to_1000.npy', allow_pickle=True)
expressions_1 = np.load('./data/top3_exps_from_0_to_1000.npy', allow_pickle=True)

In [6]:
cat_ref = references[1]
cat_exps = expressions_1[1]
print(cat_ref, cat_exps)

[['cat'], ['Cat on right'], ['cat, but not in reflection']] ['carpet' 'cat' 'dog']


In [7]:
test_scores = []
for i in cat_ref:
    score = scorer.score(i[0], cat_exps[1])
    print(score['rouge1'], score['rougeL'])
    test_scores.append(score['rouge1'])
print(np.average(test_scores))

Score(precision=1.0, recall=1.0, fmeasure=1.0) Score(precision=1.0, recall=1.0, fmeasure=1.0)
Score(precision=1.0, recall=0.3333333333333333, fmeasure=0.5) Score(precision=1.0, recall=0.3333333333333333, fmeasure=0.5)
Score(precision=1.0, recall=0.2, fmeasure=0.33333333333333337) Score(precision=1.0, recall=0.2, fmeasure=0.33333333333333337)
0.7074074074074074


In [31]:
rouge1_scores = []
rougeL_scores = []
for start in files:
    references = np.load(f'./data/references_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    expressions = np.load(f'./data/top5_exps_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    for ref, exps in zip(references, expressions):
        item_reference = [word_tokenize(r[0].lower()) for r in ref]
        exp_recalls_rouge_1 = []
        exp_recalls_rouge_L = []
        for exp in exps:
            exp_recalls_rouge_1.append(np.average([scorer.score(target[0], exp)['rouge1'].recall for target in ref]))
            exp_recalls_rouge_L.append(np.average([scorer.score(target[0], exp)['rougeL'].recall for target in ref]))
        rouge1_scores.append(max(exp_recalls_rouge_1))
        rougeL_scores.append(max(exp_recalls_rouge_L))
print(len(rouge1_scores))

5000


In [32]:
nonzero_rouge1 = [score for score in rouge1_scores if score != 0]
print(np.average(nonzero_rouge1), np.std(nonzero_rouge1))
print(np.average(rouge1_scores), np.std(rouge1_scores))
# top3 :0.24809894991750528 0.16986040769537178
#0.1687072859439036 0.18169670240599997

0.24571196905987616 0.16725547757380804
0.18330112891866762 0.17974669943941723


In [10]:
nonzero_rougeL = [score for score in rougeL_scores if score != 0]
print(np.average(nonzero_rougeL), np.std(nonzero_rougeL))
print(np.average(rougeL_scores), np.std(rougeL_scores))

0.2480695381527994 0.1698341010559976
0.1686872859439036 0.18167124031241783


In [70]:


references = np.load('./data/references_from_0_to_1000.npy', allow_pickle=True)
expressions_1 = np.load('./data/top3_exps_from_0_to_1000.npy', allow_pickle=True)

#### Add meteor metric (include synonims)
#### Improve matching algorithm (prioritizing matched objects with synonims)
#### Add relations to RSA ?
#### Generate attributes/objects/relations from scenegraph on the training dataset.
#### Learn threshold (entropy & for relations?) from the training data
#### Idea: treating nodes as objects & edges as relations. -> generate expressions.

In [33]:
# FINDING ALL IMAGES WITH 0 SCORE EXPRESSION
bad_bleu_score_images = []
for i in range(len(bleu_score)):
    if bleu_score[i] == 0:
        bad_bleu_score_images.append(i)
print(len(bad_bleu_score_images))
print(bad_bleu_score_images[:10])

bad_rouge1_score_images = []
bad_rougeL_score_images = []
for i in range(len(rouge1_scores)):
    if rouge1_scores[i] == 0:
        bad_rouge1_score_images.append(i)
    if rougeL_scores[i] == 0:
        bad_rougeL_score_images.append(i)
print(len(bad_rouge1_score_images))
print(bad_rouge1_score_images[:10])
print(len(bad_rougeL_score_images))
print(bad_rougeL_score_images[:10])

1396
[12, 14, 24, 27, 30, 35, 36, 42, 43, 46]
1270
[12, 14, 24, 27, 30, 35, 36, 42, 43, 46]
1270
[12, 14, 24, 27, 30, 35, 36, 42, 43, 46]


In [34]:
rouge_bad_intersection = np.intersect1d(bad_rougeL_score_images, bad_bleu_score_images)
print(len(rouge_bad_intersection))
bleu_rouge_bad_intersection = np.intersect1d(bad_rougeL_score_images, bad_bleu_score_images)
print(len(bleu_rouge_bad_intersection))
bleu_rouge_bad_union = np.union1d(bad_rougeL_score_images, bad_bleu_score_images)
print(len(bleu_rouge_bad_union))

1270
1270
1396


In [13]:
np.save('bad_rouge_score_reference_indexes.npy',bad_rouge1_score_images)
np.save('bad_bleu_score_reference_indexes.npy',bad_bleu_score_images)

In [48]:
from nltk.translate.meteor_score import meteor_score as meteor 

In [49]:
meteor(ref_1, "person elephant")

0.0

In [13]:
meteor_score = []
files = [0,1000,2000,3000,4000]
for start in files:
    references = np.load(f'./data/references_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    expressions = np.load(f'./data/top5_exps_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    for ref, exps in zip(references, expressions):
        item_reference = [r[0].lower() for r in ref]
    #     print(item_reference)
    #     print(exps)
    #     print('$$$$$$$$$$$')
        best_matched = max([meteor(item_reference, exp) for exp in exps])
        meteor_score.append(best_matched)
print(len(meteor_score))

5000


In [14]:
x = [score for score in meteor_score if score != 0]
print(np.average(x), np.std(x))
print(np.average(meteor_score), np.std(meteor_score))
print(len(x))

0.2535472318146614 0.13964477796877744
0.19158028835915816 0.1631145199695016
3778


### DUMPING LABELS TO JSON

In [5]:
files = [0,1000,2000,3000,4000]
exps = {}
for start in files:
    references = np.load(f'./data/test/detectron2_with_target/references_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    expressions = np.load(f'./data/test/detectron2_with_target/top3_exps_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    for i, expressions in enumerate(expressions):
        exps[start+i] = expressions[0]


In [7]:
exps[500]

'blue jar'

In [8]:
with open('generated_expressions_with_true_target.json', 'w') as fp:
    json.dump(exps, fp)

In [23]:
bleu_score = []
counter = 0

references = np.load(f'./data/test/references_from_0_to_5000.npy', allow_pickle=True)
expressions = np.load(f'./data/test/top3_exps_from_0_to_5000.npy', allow_pickle=True)
for ref, exps in zip(references, expressions):
    item_reference = [word_tokenize(r[0].lower()) for r in ref]
#     print(item_reference)
#     print(exps)
#     print('$$$$$$$$$$$')

    #best_matched = max([sentence_bleu(item_reference, word_tokenize(exp)) for exp in exps])#[0]
    best_matched = sentence_bleu(item_reference, word_tokenize(exps[0]))
    bleu_score.append(best_matched)
print(len(bleu_score))

/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


5000


In [25]:
non_zero_entries = [score for score in bleu_score if score != 0]
print(np.average(non_zero_entries), np.std(non_zero_entries))
print(np.average(bleu_score), np.std(bleu_score))
print(len(non_zero_entries))

0.6618485505995223 0.25576376768967624
0.30034687226206314 0.37183500775597955
2269


In [45]:
rouge1_scores = []
rougeL_scores = []
references = np.load(f'./data/generated_expressions/test/generated_expression_binary_utterance_lstm_adjusted/references_from_0_to_5000.npy', allow_pickle=True)
expressions = np.load(f'./data/generated_expressions/test/generated_expression_binary_utterance_lstm_adjusted/top3_exps_from_0_to_5000.npy', allow_pickle=True)
for ref, exps in zip(references, expressions):
    item_reference = [word_tokenize(r[0].lower()) for r in ref]
    exp_recalls_rouge_1 = []
    exp_recalls_rouge_L = []
    for exp in exps:
        exp_recalls_rouge_1.append(np.average([scorer.score(target[0], exp)['rouge1'].recall for target in ref]))
        exp_recalls_rouge_L.append(np.average([scorer.score(target[0], exp)['rougeL'].recall for target in ref]))
    rouge1_scores.append(max(exp_recalls_rouge_1))
    rougeL_scores.append(max(exp_recalls_rouge_L))
print(np.average(rouge1_scores), np.std(rouge1_scores))
print(np.average(rougeL_scores), np.std(rougeL_scores))

0.18100858265645028 0.19190863322770094
0.17744893184429947 0.18789276425550427


In [50]:
meteor_score = []

references = np.load(f'./data/generated_expressions/test/generated_expression_binary_utterance_lstm_adjusted/references_from_0_to_5000.npy', allow_pickle=True)
expressions = np.load(f'./data/generated_expressions/test/generated_expression_binary_utterance_lstm_adjusted/top3_exps_from_0_to_5000.npy', allow_pickle=True)
for ref, exps in zip(references, expressions):
    item_reference = [r[0].lower() for r in ref]
#     print(item_reference)
#     print(exps)
#     print('$$$$$$$$$$$')
    best_matched = max([meteor(item_reference, exp) for exp in exps])
    meteor_score.append(best_matched)
print(len(meteor_score))
x = [score for score in meteor_score if score != 0]
print(np.average(x), np.std(x))
print(np.average(meteor_score), np.std(meteor_score))
print(len(x))

5000
0.25260105199765515 0.15058741794027786
0.1787910246039403 0.1710176803215331
3539


In [28]:
f = open('generated_expression_probabilistic_utterance_lstm.json')
dat = json.load(f)
long_exps = 0
for i in range(5000):
    if " " in dat[str(i)]:
        long_exps += 1
print(long_exps)

40


In [31]:
exps = {}
long_exps = 0
expressions = np.load(f'./data/test/top3_exps_from_0_to_5000.npy', allow_pickle=True)
for i, expressions in enumerate(expressions):
    exps[i] = expressions[0]
    if " " in expressions[0]:
        long_exps += 1
print(long_exps)

3577


In [40]:
with open('generated_expressions.json', 'w') as fp:
    json.dump(exps, fp)

In [41]:
bleu_score = []
files = [0,1000,2000,3000,4000]
for start in files:
    references = np.load(f'./data/test/references_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    expressions = np.load(f'./data/test/top3_exps_from_{start}_to_{start+1000}.npy', allow_pickle=True)
    for ref, exps in zip(references, expressions):
        item_reference = [word_tokenize(r[0].lower()) for r in ref]
    #     print(item_reference)
    #     print(exps)
    #     print('$$$$$$$$$$$')

#         best_matched = max([sentence_bleu(item_reference, word_tokenize(exp)) for exp in exps])#[0]
        best_matched = sentence_bleu(item_reference, word_tokenize(exps[0]))
        bleu_score.append(best_matched)
print(len(bleu_score))

/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/scratch2/hle/py3_env/lib/python3.7/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


5000


In [42]:
non_zero_entries = [score for score in bleu_score if score != 0]
print(np.average(non_zero_entries), np.std(non_zero_entries))
print(np.average(bleu_score), np.std(bleu_score))
print(len(non_zero_entries))

0.6629174540870093 0.25348291217850427
0.3008319406646848 0.3715981234667934
2269


### EVALUATE WITH REFCOCO API

In [2]:
expressions = np.load(f'./data/generated_expressions/test/generated_expression_binary_utterance_lstm_adjusted/top3_exps_from_0_to_5000.npy', allow_pickle=True)

In [5]:
expressions[137]

['the left woman', 'brown man', 'purple shirt']

In [8]:
import sys
sys.path.append("/scratch2/hle/git/refer")
from refer import REFER

In [9]:
refer = REFER("/scratch2/hle/git/refer/data","refcoco", "google")

loading dataset refcoco into memory...
creating index...
index created.
DONE (t=6.39s)


In [ ]:
5*5

In [1]:
import sys
sys.path.append("/scratch2/hle/git/refer")
sys.path.append("/scratch2/hle/git/rsa_refer")
sys.path.append('/scratch2/hle/git/refer/evaluation')
sys.path.append('/scratch2/hle/git/refer/evaluation/bleu')
sys.path.append('/scratch2/hle/git/refer/evaluation/cider')
sys.path.append('/scratch2/hle/git/refer/evaluation/meteor')
sys.path.append('/scratch2/hle/git/refer/evaluation/rouge')
sys.path.append('/scratch2/hle/git/refer/evaluation/tokenizer')
from refEvaluation import RefEvaluation
import json
import numpy as np
import pandas as pd
import os
from refer import REFER
refer = REFER('/scratch2/hle/git/refer/data', 'refcoco', 'google')


loading dataset refcoco into memory...
creating index...
index created.
DONE (t=11.82s)


In [4]:
with open('config.json') as config_file:
    config = json.load(config_file)
data_path = config['data_path']
f = open('generated_expressions.json')
expressions = json.load(f)#np.load('/scratch2/hle/git/rsa_refer/data/generated_expressions/test/generated_expression_binary_utterance_lstm_adjusted/top3_exps_from_0_to_5000.npy', allow_pickle=True)
eval_expressions = []
for i in range(5000):
    df = pd.read_csv(os.path.join(data_path,'refCOCO/test/attr_tables/attr_'+str(i)+'.tsv'), encoding='utf-8',sep='\t')

    ref_id = df['ref_id'][0]
    eval_expressions.append({'sent': expressions[str(i)], 'ref_id':ref_id})
    if i % 100 == 0:
        print('done', i)


('done', 0)
('done', 100)
('done', 200)
('done', 300)
('done', 400)
('done', 500)
('done', 600)
('done', 700)
('done', 800)
('done', 900)
('done', 1000)
('done', 1100)
('done', 1200)
('done', 1300)
('done', 1400)
('done', 1500)
('done', 1600)
('done', 1700)
('done', 1800)
('done', 1900)
('done', 2000)
('done', 2100)
('done', 2200)
('done', 2300)
('done', 2400)
('done', 2500)
('done', 2600)
('done', 2700)
('done', 2800)
('done', 2900)
('done', 3000)
('done', 3100)
('done', 3200)
('done', 3300)
('done', 3400)
('done', 3500)
('done', 3600)
('done', 3700)
('done', 3800)
('done', 3900)
('done', 4000)
('done', 4100)
('done', 4200)
('done', 4300)
('done', 4400)
('done', 4500)
('done', 4600)
('done', 4700)
('done', 4800)
('done', 4900)


In [5]:
refEval = RefEvaluation(refer, eval_expressions)
refEval.evaluate()


tokenization...
setting up scorers...
computing Bleu score...
{'reflen': 13272, 'guess': [10533, 5533, 1956, 1063], 'testlen': 10533, 'correct': [2508, 52, 2, 0]}
ratio: 0.793625678119
Bleu_1: 0.184
Bleu_2: 0.036
Bleu_3: 0.010
Bleu_4: 0.000
computing METEOR score...
METEOR: 0.079
computing Rouge score...
ROUGE_L: 0.197
computing CIDEr score...
CIDEr: 0.317
